
# require_recipient函数

函数在`action.codon`中的声明如下：

In [ ]:
def require_recipient(account: Name):

`require_recipient`函数用来通知其它合约. 如果account合约有相同的action，那么这个action将被调用。

以下的`sender.codon`, `receiver.codon`的代码演示了如何从一个合约发送通知到另一个合约。

In [4]:
!mkdir -p tests/notify

In [5]:
%%writefile tests/notify/sender.codon

from chain.name import Name
from chain.contract import Contract
from chain.action import require_recipient

@contract(main=True)
class MyContract(Contract):

    @action('sayhello')
    def sayhello(self, receiver: Name):
        print('hello, world')
        require_recipient(receiver)

Writing notify/sender.codon


In [6]:
%%writefile tests/notify/receiver.codon

from chain.name import Name
from chain.contract import Contract

@contract(main=True)
class MyContract(Contract):

    @action('sayhello', notify=True)
    def sayhello(self, receiver: Name):
        assert not self.receiver == self.first_receiver
        assert receiver == self.receiver
        print('hello, world from notify')

Writing notify/receiver.codon


这里，要注意的是，`receiver.codon`中的`sayhello`函数和`sender.codon`中的`sayhello`函数的定义有些不同，`receiver.codon`中的`sayhello`的`action`decorator中多了`notify=True`，这是用来指定这个action是一个用来接收通知的action，只能通过调用`require_recipient`来触发。

以下是测试代码：

In [7]:
%%writefile tests/test.py

import os
from ipyeos.chaintester import ChainTester

from pscdk.tester import update_auth, logger

def init_notify(dir_name=''):
    t = ChainTester(True)
    update_auth(t, 'hello')

    wasm_file = os.path.join(dir_name, 'notify/sender.wasm')
    with open(wasm_file, 'rb') as f:
        code = f.read()
    abi_file = os.path.join(dir_name, 'notify/sender.abi')
    with open(abi_file, 'r') as f:
        abi = f.read()
    t.deploy_contract('hello', code, abi)
    t.produce_block()

    wasm_file = os.path.join(dir_name, 'notify/receiver.wasm')
    with open(wasm_file, 'rb') as f:
        code = f.read()
    abi_file = os.path.join(dir_name, 'notify/receiver.abi')
    with open(abi_file, 'r') as f:
        abi = f.read()
    t.deploy_contract('alice', code, abi)
    t.produce_block()
    return t

def test_notify():
    t = init_notify()
    args = {'receiver': 'alice'}
    ret = t.push_action('hello', 'sayhello', args, {'hello': 'active'})
    t.produce_block()
    logger.info("++++++++++%s\n", ret['elapsed'])

Overwriting test.py


编译：

In [8]:
%%bash
python-contract build notify/receiver.codon
python-contract build notify/sender.codon

 class.cpp 249 +++++++class name: MyContract
 class.cpp 26 generating apply method for class MyContract
 class.cpp 53 +++++++fn.name: sayhello
 typecheck.cpp 76 +++++++ctx->cache->classRealizationCnt: 527
 compiler.cpp 149 Generating abi file: notify/receiver.abi
 llvisitor.cpp 495 ++++++++++executable dir:/Users/newworld/dev/github/codon/build


/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/bin/codon build --release  --march=wasm32 --linker-flags=" -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib/codon -lcodonrt-wasm32 -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib -lc++ -lc" notify/receiver.codon


 class.cpp 249 +++++++class name: MyContract
 class.cpp 26 generating apply method for class MyContract
 class.cpp 53 +++++++fn.name: sayhello
 typecheck.cpp 76 +++++++ctx->cache->classRealizationCnt: 511
 compiler.cpp 149 Generating abi file: notify/sender.abi
 llvisitor.cpp 495 ++++++++++executable dir:/Users/newworld/dev/github/codon/build


/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/bin/codon build --release  --march=wasm32 --linker-flags=" -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib/codon -lcodonrt-wasm32 -L/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/pscdk/codon/lib -lc++ -lc" notify/sender.codon


测试：

In [9]:
%%bash
ipyeos -m pytest -s -x tests/test.py -k test_notify

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.2.0, pluggy-1.0.0
benchmark: 4.0.0 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /Users/newworld/dev/github/pscdk-book/notebook-zh
plugins: asyncio-0.20.3, integration-0.2.3, dash-2.9.3, mock-3.10.0, cov-4.0.0, anyio-3.6.2, benchmark-4.0.0
asyncio: mode=strict
collected 1 item



info  2023-05-12T08:18:58.469 thread-0  chain_manager.cpp:117         operator()           ] Support for builtin protocol feature 'GET_BLOCK_NUM' (with digest of '35c2186cc36f7bb4aeaf4487b36e57039ccf45a9136aa856a5d569ecca55ef2b') is enabled with preactivation required
info  2023-05-12T08:18:58.469 thread-0  chain_manager.cpp:227         operator()           ] Saved default specification for builtin protocol feature 'GET_BLOCK_NUM' (with digest of '35c2186cc36f7bb4aeaf4487b36e57039ccf45a9136aa856a5d569ecca55ef2b') to: /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tmpqo17h_d2/protocol_features/BUILTIN-GET_BLOCK_NUM.json
info  2023-05-12T08:18:58.470 thread-0  chain_manager.cpp:117         operator()           ] Support for builtin protocol feature 'CRYPTO_PRIMITIVES' (with digest of '6bcb40a24e49c26d0a60513b6aeb8551d264e4717f306b81a37a5afb3b47cedc') is enabled with preactivation required
info  2023-05-12T08:18:58.470 thread-0  chain_manager.cpp:227         operator()           ] Saved

info  2023-05-12T08:18:58.474 thread-0  chain_manager.cpp:117         operator()           ] Support for builtin protocol feature 'DISALLOW_EMPTY_PRODUCER_SCHEDULE' (with digest of '68dcaa34c0517d19666e6b33add67351d8c5f69e999ca1e37931bc410a297428') is enabled with preactivation required
info  2023-05-12T08:18:58.475 thread-0  chain_manager.cpp:227         operator()           ] Saved default specification for builtin protocol feature 'DISALLOW_EMPTY_PRODUCER_SCHEDULE' (with digest of '68dcaa34c0517d19666e6b33add67351d8c5f69e999ca1e37931bc410a297428') to: /var/folders/pv/678xdw3s56xggsvz36r65kzr0000gp/T/tmpqo17h_d2/protocol_features/BUILTIN-DISALLOW_EMPTY_PRODUCER_SCHEDULE.json
info  2023-05-12T08:18:58.475 thread-0  chain_manager.cpp:117         operator()           ] Support for builtin protocol feature 'FIX_LINKAUTH_RESTRICTION' (with digest of 'e0fb64b1085cc5538970158d05a009c24e276fb94e1a0bf6a528b48fbc4ff526') is enabled with preactivation required
info  2023-05-12T08:18:58.475 thre

test.py .

============================== 1 passed in 0.98s ===============================
export CHAIN_API_LIB=/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/lib/libchain_api.dylib
export VM_API_LIB=/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/lib/libvm_api.dylib
export PYTHON_SHARED_LIB_PATH=/Users/newworld/miniconda3/envs/chat/lib/libpython3.10.dylib
/Users/newworld/miniconda3/envs/chat/lib/python3.10/site-packages/ipyeos/release/bin/ipyeos -m pytest -s -x test.py -k test_notify


输出：

```
[(hello,sayhello)->hello]: CONSOLE OUTPUT BEGIN =====================
hello, world

[(hello,sayhello)->hello]: CONSOLE OUTPUT END   =====================
debug 2023-03-28T13:08:01.110 thread-0  apply_context.cpp:30          print_debug          ] 
[(hello,sayhello)->alice]: CONSOLE OUTPUT BEGIN =====================
hello, world from notify

[(hello,sayhello)->alice]: CONSOLE OUTPUT END   =====================
```